In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, ZeroPadding2D, GlobalAvgPool2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import tensorflow as tf

In [ ]:
'''
encoder = LabelEncoder()

train_csv_encode = train_csv
train_csv_encode.Class = encoder.fit_transform(train_csv_encode.Class)
print(train_csv_encode.head())
csv = train_csv_encode.to_csv('/home/tim/Datasets/faces_age_dataset/Train.csv', index=False)
'''

In [2]:
train_images_path = '/home/tim/Datasets/faces_age_dataset/Train'
train_csv_path = '/home/tim/Datasets/faces_age_dataset/Train.csv'
train_csv = pd.read_csv(train_csv_path)

In [3]:
train_csv['Class'] = train_csv['Class'].astype('string')

In [4]:

train_datagen = ImageDataGenerator(rescale=1 / 127.5, validation_split=0.2,)
"""
                                   rotation_range=45, zoom_range=0.2,
                                   horizontal_flip=True, vertical_flip=True, fill_mode='nearest', shear_range=0.1,
                                   height_shift_range=0.1, width_shift_range=0.1
"""


"\n                                   rotation_range=45, zoom_range=0.2,\n                                   horizontal_flip=True, vertical_flip=True, fill_mode='nearest', shear_range=0.1,\n                                   height_shift_range=0.1, width_shift_range=0.1\n"

In [5]:
train_generator = train_datagen.flow_from_dataframe(train_csv,
                                                    directory=train_images_path,
                                                    subset='training',
                                                    x_col='ID',
                                                    y_col='Class',
                                                    target_size=(160, 160),
                                                    class_mode='sparse',)

val_generator = train_datagen.flow_from_dataframe(train_csv,
                                                  directory=train_images_path,
                                                  subset='validation',
                                                  x_col='ID',
                                                  y_col='Class',
                                                  target_size=(160, 160),
                                                  class_mode='sparse',)

Found 15925 validated image filenames belonging to 3 classes.
Found 3981 validated image filenames belonging to 3 classes.


In [ ]:
"""
model = Sequential()
model.add(ZeroPadding2D((2, 2), input_shape=(64, 64, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.2))

model.add(ZeroPadding2D((2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.3))

model.add(ZeroPadding2D((2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.3))

model.add(Flatten())
"""

In [6]:
base_model = MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
feature_batch.shape

TensorShape([32, 5, 5, 1280])

In [ ]:
global_average_layer = GlobalAvgPool2D()
feature_batch_average = global_average_layer(feature_batch)

In [ ]:
prediction_layer = Dense(3)
prediction_batch_layer = prediction_layer(feature_batch_average)

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
callback = [EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
            TensorBoard(log_dir='/home/tim/trained/age_pred_log')]

In [ ]:
model.fit(train_generator, validation_data=val_generator, epochs=5, batch_size=128, callbacks=[callback])

In [ ]:
model.save('/home/tim/trained/age_prediction')